In [125]:
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 237 kB 7.9 MB/s 


In [132]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.0 MB/s 


In [134]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import csv
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import regularizers
import tensorflow_model_optimization as tfmot
import tensorflow_addons as tfa

In [99]:
with open("eng-ind.csv") as f:
    csvreader = csv.reader(f, delimiter=",")
    next(csvreader)
    eng_ind, label = list(),list()
    for i in csvreader:
        eng_ind.append(i[0])
        label.append(0)
        eng_ind.append(i[1])
        label.append(1)
        
vectorize = TextVectorization(output_sequence_length=32)
vectorize.adapt(tf.data.Dataset.from_tensor_slices(eng_ind))


In [100]:
datas = list()
for i in eng_ind:
  datas.append(vectorize(i))

In [101]:
dataset = tf.data.Dataset.from_tensor_slices((datas,label)).batch(8)

In [102]:
train = dataset.take(int(0.9*len(dataset)))
val = dataset.skip(int(0.9*len(dataset)))

In [103]:
len(vectorize.get_vocabulary())

8838

In [141]:
model = tf.keras.Sequential()

# model.add(tfa.layers.EmbeddingBag(input_dim=1000, output_dim=64, input_shape=[32]))
model.add(layers.Embedding(input_dim=1000, output_dim=64, input_shape=[32]))
model.add(layers.Flatten())

model.add(layers.Dense(64, activation="tanh"))
                      #  kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                      # bias_regularizer=regularizers.L2(1e-4),
                      # activity_regularizer=regularizers.L2(1e-5)))
model.add(layers.Dense(64,activation="tanh"))
                      #  kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                      # bias_regularizer=regularizers.L2(1e-4),
                      # activity_regularizer=regularizers.L2(1e-5)))
model.add(layers.Dense(32,activation="tanh"))
                      #  kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                      # bias_regularizer=regularizers.L2(1e-4),
                      # activity_regularizer=regularizers.L2(1e-5)))

model.add(layers.Dense(1, activation="sigmoid"))

model.compile(
    loss = "binary_crossentropy",
    optimizer = "RMSprop",
    metrics = ["accuracy"]
)

model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 32, 64)            64000     
                                                                 
 flatten_6 (Flatten)         (None, 2048)              0         
                                                                 
 dense_44 (Dense)            (None, 64)                131136    
                                                                 
 dense_45 (Dense)            (None, 64)                4160      
                                                                 
 dense_46 (Dense)            (None, 32)                2080      
                                                                 
 dense_47 (Dense)            (None, 1)                 33        
                                                                 
Total params: 201,409
Trainable params: 201,409
Non-t

In [106]:
model.fit(train,
          validation_data=val,
          epochs=5)

Epoch 1/5
1984/1984 [==============================] - 8s 4ms/step - loss: 0.0470 - accuracy: 0.9791 - val_loss: 0.0078 - val_accuracy: 0.9989
Epoch 2/5
1984/1984 [==============================] - 8s 4ms/step - loss: 0.0178 - accuracy: 0.9941 - val_loss: 0.0028 - val_accuracy: 0.9989
Epoch 3/5
1984/1984 [==============================] - 8s 4ms/step - loss: 0.0151 - accuracy: 0.9950 - val_loss: 0.0129 - val_accuracy: 0.9977
Epoch 4/5
1984/1984 [==============================] - 8s 4ms/step - loss: 0.0184 - accuracy: 0.9946 - val_loss: 0.0304 - val_accuracy: 0.9949
Epoch 5/5
1984/1984 [==============================] - 8s 4ms/step - loss: 0.0166 - accuracy: 0.9946 - val_loss: 0.0088 - val_accuracy: 0.9983


## Predict

## Greater than 0.5 mean Indonesian language, less than 0.5 mean English language.

In [107]:
teks = tf.constant(["i love you",
               "When using this layer as the first layer in a model, provide an input_shape argument (tuple of integers or None, e.g. (10, 128) for sequences of 10 vectors of 128-dimensional vectors, or (None, 128) for variable-length sequences of 128-dimensional vectors.",
               "Indonesia, dengan nama resmi Republik Indonesia atau lengkapnya Negara Kesatuan Republik Indonesia, adalah sebuah negara kepulauan di Asia Tenggara yang dilintasi garis khatulistiwa dan berada di antara daratan benua Asia dan Oseania, serta antara Samudra Pasifik dan Samudra Hindia"])


model.predict(vectorize(teks))

array([[5.3625766e-09],
       [5.4949525e-09],
       [1.0000000e+00]], dtype=float32)

## Save model into tensorflow format

In [108]:
model.save("model")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


## Convert tensorflow model to tflite

In [109]:
converter = tf.lite.TFLiteConverter.from_saved_model("model")
tflite_model = converter.convert()

with open("model.tflite","wb") as f:
    f.write(tflite_model)

In [115]:
teks = tf.constant(["i love you",
               "When using this layer as the first layer in a model, provide an input_shape argument (tuple of integers or None, e.g. (10, 128) for sequences of 10 vectors of 128-dimensional vectors, or (None, 128) for variable-length sequences of 128-dimensional vectors.",
               "Indonesia, dengan nama resmi Republik Indonesia atau lengkapnya Negara Kesatuan Republik Indonesia, adalah sebuah negara kepulauan di Asia Tenggara yang dilintasi garis khatulistiwa dan berada di antara daratan benua Asia dan Oseania, serta antara Samudra Pasifik dan Samudra Hindia"])


In [116]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# print(input_details)
# print(output_details)

# must have 2 dimension and float32 dtype
for i in range(len(teks)):
  input_data = np.array([vectorize(teks[i])], dtype=np.float32)

  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()

  predictions = interpreter.get_tensor(output_details[0]['index'])

  print(teks[i])
  print(predictions)

tf.Tensor(b'i love you', shape=(), dtype=string)
[[5.3625766e-09]]
tf.Tensor(b'When using this layer as the first layer in a model, provide an input_shape argument (tuple of integers or None, e.g. (10, 128) for sequences of 10 vectors of 128-dimensional vectors, or (None, 128) for variable-length sequences of 128-dimensional vectors.', shape=(), dtype=string)
[[0.00478767]]
tf.Tensor(b'Indonesia, dengan nama resmi Republik Indonesia atau lengkapnya Negara Kesatuan Republik Indonesia, adalah sebuah negara kepulauan di Asia Tenggara yang dilintasi garis khatulistiwa dan berada di antara daratan benua Asia dan Oseania, serta antara Samudra Pasifik dan Samudra Hindia', shape=(), dtype=string)
[[0.00478767]]
